### This code extracts data about 1) spike counts for each trial, separated into correct/incorrect contralateral, ipsilateral, and nogo 2) percent corrent choices for each session of behavior and 3) average time from stimulus presentation to auditory GO cue.

In [2]:
## to run this code, you need to have the behavioral data dictionaries previously loaded in
## see 'loading_in_tar' and 'loading_in_npy' files in repo, limited code copied below:

# os.chdir('/your/filepath/here')
# goCue_times = np.load("goCue_times.npy", allow_pickle=True)
# response_choices = np.load('response_choices.npy', allow_pickle=True)
# feedback_types = np.load('feedback_types.npy', allow_pickle=True)
# stim_times = np.load('stim_times.npy', allow_pickle=True)

In [ ]:
sessions = np.arange(1, 40)
# initialize empty array and dictionary to populate with behavioral data
perc_corr = np.zeros(40)
trial_type = {new_key: [] for new_key in sessions}

for s in sessions:

    trials = len(goCue_times[s]) # nTrials
    contra_Ctrials = np.zeros(trials) # to populate with trials where *correct response is *contraleteral to probe
    contra_Itrials = np.zeros(trials) # ' ' *incorrect response is *contraleteral to probe
    ipsi_Ctrials = np.zeros(trials) # '  ' *correct response is *ipsileteral to probe
    ipsi_Itrials = np.zeros(trials) # '  ' *correct response is *ipsileteral to probe
    nogo_trials = np.zeros(trials) # '  ' *NoGo trials
    
    # for each trial, classify (by marking as '1' instead of '0') which trial type it belongs to
    for t in range(trials):
        response = int(response_choices[s][t]) # -1 right, 0 nogo, 1 left
        feedback = int(feedback_types[s][t]) # -1 incorrect, 1 correct
        if response == 1 and feedback == 1:
            contra_Ctrials[t] = 1
        elif response == 1 and feedback == -1:
            contra_Itrials[t] = 1
        elif response == -1 and feedback == 1:
            ipsi_Ctrials[t] = 1
        elif response == -1 and feedback == -1:
            ipsi_Itrials[t] = 1
        elif response == 0:
            nogo_trials[t] = 1
    
    # calculate percent correct and add to list
    pc = np.count_nonzero(feedback_types[s]==1)/(len(feedback_types[s]))
    perc_corr[s] = pc
    
    # put all arrays of trial types into a single dictionary
    # to be indexed as trial_type[session][trial type]
    # trial type index corresponds with it's index in the stack below
    trial_type[s] = np.stack((contra_Ctrials, contra_Itrials, ipsi_Ctrials, ipsi_Itrials, nogo_trials), axis=0)
    
    ## now that we know which trials are which type, we need timestamps for each of those trials to get FR
    contra_corr_SC = []
    contra_corr_nBins = []
    contra_incorr_SC = []
    contra_incorr_nBins = []
    
    ipsi_corr_SC = []
    ipsi_corr_nBins = []
    ipsi_incorr_SC = []
    ipsi_incorr_nBins = []
    
    nogo_SC = []
    nogo_nBins = []
    
    time_window = []
    
    for t in range(trials):
        
        # get our start and end times for the period of interest
        lowerT = stim_times[s][t]
        upperT = goCue_times[s][t]
        this_trial = []
        
        for key,value in vis_timestamps.items():
            if s == key:
                all_stamps = np.arange(1, len(vis_timestamps[s]))
                for i in all_stamps:
                    if lowerT <= vis_timestamps[s][i] < upperT: 
                        this_trial.append(vis_timestamps[s][i])
        time_window.append(upperT-lowerT)
        
        if trial_type[s][0][t] == 1: ## if this contra trial was correct
            contra_corr_SC.append(len(this_trial))
            contra_corr_nBins.append((upperT-lowerT)//0.10)
            
        elif trial_type[s][0][t] == 1: ## if this contra trial was incorrect
            contra_incorr_SC.append(len(this_trial))
            contra_incorr_nBins.append((upperT-lowerT)//0.10)

        elif trial_type[s][2][t] == 1: ## if this ipsi trial was correct
            ipsi_corr_SC.append(len(this_trial))
            ipsi_corr_nBins.append((upperT-lowerT)//0.10)
            
        elif trial_type[s][3][t] == 1: ## if this ipsi trial was incorrect
            ipsi_incorr_SC.append(len(this_trial))
            ipsi_incorr_nBins.append((upperT-lowerT)//0.10)
            
        elif trial_type[s][4][t] == 1: ## if this was a nogo trial
            nogo_SC.append(len(this_trial))
            nogo_nBins.append((upperT-lowerT)//0.10)